In [1]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base, vesting_schedule_translate


params_base["FP Mine Block Boundary Action"] = "TEST Mine Block Boundary Action"
state_base["K Qi"] = 1
state_base["K Quai"] = 500


ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)


Placeholder has no control action options
Mezzanine Wiring Passthrough has no policy options
No python implementation for Controller Update Policy / Linear Controller Policy. To fix this, go to Implementations/Python/Policies and add Linear Controller Policy
Beta Estimation Policy has no policy options
No python implementation for Increment Block Number Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Increment Block Number Mechanism
No python implementation for Update Historical Mined Ratio Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Mined Ratio Mechanism
No python implementation for Update Historical Qi Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Qi Hash Mechanism
No python implementation for Update Historical Quai Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Quai Hash Mechanism
No python implementation for Log Simul

In [2]:
# ADD AN INSPECT CODE STEP TO SHOW IMPLEMENTATIONS
# WAIT FOR MSML IMPLEMENTATION OF IT

In [3]:
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])

spaces = []
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()


spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 1000,
  'Blocks to Mine': [{'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000},
                     {'Difficulty': 6000}]}]



[{'Block Difficulty': [6000,
                       6000,
                       6000,
                       6000,
                       6000,
                       6000,
                       6000,
                       6000,
                       6000,
               

In [4]:
assert False, "Open issue to fix up hashpower comparisons with change in model"

AssertionError: Open issue to fix up hashpower comparisons with change in model

# Hashpower Comparisons

The current dummy model multiplies the Qi price by 1000. We display a low, medium and high scenario below to illustrate when hashpower is not enough, just right and too high and how difficulty adjusts as well as what the spaces look like.

The target mining time is set to 2 seconds.

## Low Hashpower

You can see that a prime block is not mined in the first epoch leading to a decrease in difficulty and two total epochs.

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 5
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 5000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Type

## Medium Hashpower

One mining epoch happens and within a reasonable time so no difficulty adjustment

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 10
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 10000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Typ

## High Hashpower

One mining epoch happens but too fast so difficulty is adjusted to be higher

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 100
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 100000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Ty